In [1]:
from utils import to_csv_1, randomTrips, shouldContinueSim, quickstart
import os
import sys
from sumolib import checkBinary
import traci

if 'SUMO_HOME' in os.environ:
    print('SUMO_HOME found')
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))

sumoBinary = checkBinary('sumo')
roadNetwork = "./config/osm.sumocfg"
sumoCmd = [sumoBinary, "-c", roadNetwork, "--start", "--quit-on-end"]

SUMO_HOME found
SUMO_HOME found


In [2]:
randomTrips(dur=800, density=1.5)

Success.


In [3]:
def get_vehicle_count():
    quickstart(sumoCmd)
    n = 0
    dict = {}
    while shouldContinueSim():
        traci.simulationStep()
        for vehicle in traci.vehicle.getIDList():
                if vehicle not in dict:
                    dict[vehicle] = 1
                    n += 1
    traci.close()
    return n, list(dict.keys())
n, all_vehicle_ids = get_vehicle_count()

 Retrying in 1 seconds
***Starting server on port 41463 ***
Loading net-file from './config/osm.net.xml.gz' ... done (108ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
Simulation ended at time: 1557.00
Reason: TraCI requested termination.
Performance: 
 Duration: 0.26s
 TraCI-Duration: 0.12s
 Real time factor: 5942.75
 UPS: 175694.656489
Vehicles: 
 Inserted: 89
 Running: 0
 Waiting: 0
Statistics (avg of 89):
 RouteLength: 4885.02
 Speed: 9.58
 Duration: 517.21
 WaitingTime: 17.72
 TimeLoss: 70.09
 DepartDelay: 0.47



In [4]:
def get_trajectory_with_time(vehicle_ids):
    quickstart(sumoCmd)
    dict = {}
    for vehicle in vehicle_ids:
        dict[vehicle] = [-1, -1]
    while shouldContinueSim():
        traci.simulationStep()
        current_time = traci.simulation.getTime()
        curr_vehicle_ids = traci.vehicle.getIDList()
        for vehicle in curr_vehicle_ids:
            x, y = traci.vehicle.getPosition(vehicle)
            dict[vehicle] += [x, y]
        for key in dict:
            if len(dict[key]) / 2 < current_time + 1:
                dict[key] += [-1, -1]
    traci.close()
    return dict

complete_trajectory = get_trajectory_with_time(all_vehicle_ids)

 Retrying in 1 seconds
***Starting server on port 55865 ***
Loading net-file from './config/osm.net.xml.gz' ... done (109ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
Simulation ended at time: 1557.00
Reason: TraCI requested termination.
Performance: 
 Duration: 1.04s
 TraCI-Duration: 0.88s
 Real time factor: 1491.38
 UPS: 44091.954023
Vehicles: 
 Inserted: 89
 Running: 0
 Waiting: 0
Statistics (avg of 89):
 RouteLength: 4885.02
 Speed: 9.58
 Duration: 517.21
 WaitingTime: 17.72
 TimeLoss: 70.09
 DepartDelay: 0.47



In [5]:
to_csv_1(complete_trajectory, "trajectory_time.csv")

# Check if the trajectory is correct

In [6]:
quickstart(sumoCmd)
while shouldContinueSim():
    traci.simulationStep()
    curr_vehicle_ids = traci.vehicle.getIDList()
    time = int(traci.simulation.getTime())
    for vehicle in curr_vehicle_ids:
        x, y = traci.vehicle.getPosition(vehicle)
        if x != complete_trajectory[vehicle][time * 2] or y != complete_trajectory[vehicle][time * 2 + 1]:
            print(complete_trajectory[vehicle])
            print("Vehicle: ", vehicle)
            print("Expected: ", complete_trajectory[vehicle][time * 2], complete_trajectory[vehicle][time * 2 + 1])
            print("Actual: ", x, y)
        assert x == complete_trajectory[vehicle][time * 2]
        assert y == complete_trajectory[vehicle][time * 2 + 1]

 Retrying in 1 seconds
***Starting server on port 43769 ***
Loading net-file from './config/osm.net.xml.gz' ... done (108ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
